In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

2024-02-02 14:09:26.026042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Extrait les dernières données et écrit le dataset

In [2]:
from lib.dataeng import Etl

In [3]:
etl = Etl()

df_weight_raw, df_food_raw, df_exos_raw, df_all = etl.extract_store_all()

Le fichier poids le plus récent est : data_BEN_1706861557.zip, parmi :

data_BEN_1706593344.zip
data_BEN_1706676305.zip
data_BEN_1706861557.zip
data_BEN_1706761334.zip
Le fichier food le plus récent est : File-Export-2020-08-03-to-2024-02-02.zip, parmi 

File-Export-2020-08-03-to-2024-02-01.zip
File-Export-2020-08-03-to-2024-01-31.zip
File-Export-2020-08-03-to-2024-02-02.zip
File-Export-2020-08-03-to-2024-01-30.zip
Le fichier exercices le plus récent est : polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip, parmi :

polar-user-data-export_12baa4ce-79a5-40f8-ad40-74308f7c2c26.zip
polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip


100%|██████████| 1868/1868 [01:37<00:00, 19.13it/s]


In [4]:
fullds_path = '/home/benjamin/Folders_Python/Weight/data/full_dataset.csv'

with open(fullds_path, 'r') as f:
    df_full = pd.read_csv(f, index_col=0)
    
cols_to_drop = ['Masse_Osseuse', 'Masse_Musculaire', 'Masse_Hydrique']
df_full.drop(columns=cols_to_drop, inplace=True)
df_full.dropna(inplace=True)

In [246]:
df_full.describe(include='all').transpose()

,count,mean,std,min,25%,50%,75%,max
Masse_Totale,1255.0,73.738695,7.771515,66.000000,69.264583,71.350000,73.313958,107.250000
Masse_Grasse,1255.0,12.189658,5.158892,7.541667,8.936769,10.838000,12.154286,37.190000
MG%,1255.0,0.160796,0.043932,0.112927,0.128908,0.151514,0.166685,0.346760
BMR,1255.0,1699.459209,59.247545,1626.348571,1664.177500,1679.370400,1705.931200,1933.516000
Calories,1255.0,1968.096016,458.310913,0.300000,1780.500000,1850.400000,2039.300000,5762.000000
Lipides,1255.0,52.875777,22.359903,0.000000,38.600000,49.300000,64.000000,235.900000
Glucides,1255.0,213.105657,75.631917,0.100000,168.650000,199.200000,240.950000,875.400000
Proteines,1255.0,157.086853,39.925493,0.000000,133.900000,162.000000,184.450000,267.100000
exo_duree,1255.0,2695.195995,2799.230731,0.000000,0.000000,2324.284000,4749.072000,28800.000000
exo_cals_bruts,1255.0,295.333865,295.635874,0.000000,0.000000,275.000000,497.500000,2290.000000


In [247]:
ds = df_full[['Calories', 'exo_cals_nets', 'Masse_Totale', 'Masse_Grasse']]

In [248]:
ds

,Calories,exo_cals_nets,Masse_Totale,Masse_Grasse
2020-08-09,1861.3,0.000000,107.250000,37.190000
2020-08-11,1772.4,0.000000,104.650000,33.150000
2020-08-12,1894.3,0.000000,104.350000,33.800000
2020-08-13,1822.6,0.000000,104.400000,34.340000
2020-08-14,1891.5,0.000000,104.500000,33.850000
...,...,...,...,...
2024-01-28,1821.9,294.128988,73.339167,12.444167
2024-01-29,1795.7,181.483944,73.896667,12.780000
2024-01-30,1680.9,0.000000,73.340000,12.521538
2024-01-31,1682.0,332.372089,73.571538,12.973077


### Classe génératrice de dataset

Le dataset est hyperparamétré par :
- PAST : le nombre de jours connus de l'historique
- FUTURE : le nombre de jours à venir pour la prévision

Les inputs sont les données connues, à savoir :
- dans le PAST, les calories (food et exercices), et les masses (totale, grasse) mesurées
- dans le FUTURE, les calories (food et exercices)

Les outputs, que le modèle doit apprendre, sont alors :
- dans le FUTURE, les masses (totale, grasse)

In [249]:
PAST = 7 # nombre de jours de l'horizon passé
FUTURE = 3 # nombre de jours de l'horizon de prédiction

In [250]:
class CreateDataset():
    """Generic class to generate dataset in tf.dataset format
    """
    
    def __init__(self, dataframe=None, past=None, future=None, batch_size=None):
        """Constructor

        Args:
            dataframe (dataframe, optional): dataframe with dates as index, ['Calories', 'exo_cals_nets', 'Masse_Totale', 'Masse_Grasse'] as columns. Defaults to None.
            past (_type_, optional): number of days in the past, including today. Defaults to None.
            future (_type_, optional): number of days in the future, including tomorrow. Defaults to None.
        """
        
        df = ds if dataframe is None else dataframe
        # df['Masse_Totale_J+1'] = df['Masse_Totale'].shift(-1)
        # df['Masse_Grasse_J+1'] = df['Masse_Grasse'].shift(-1)
        self.dataframe = df.dropna()
        
        self.past = PAST if past is None else past
        self.future = FUTURE if future is None else future
        self.batch_size = 32 if batch_size is None else batch_size
        
    def create_dataset(self, batch_size=32):
        """engine to create the tf.dataset

        Args:
            batch_size (int, optional): _description_. Defaults to 32.
        """
        
        dataframe = self.dataframe
        # 
        # the target for the input window at index i is the dataframe[['Masse_Totale', 'Masse_Grasse']] from date i + PAST included to date i + PAST + FUTURE - 1 included (length = FUTURE )
        
        # prepare the input data
        
        # a) all values known in the PAST : the first piece of the input window at index i is an extract of the whole dataframe from date i included to date i+PAST-1 included (length = PAST)
        data_inputs1 = dataframe[['Calories', 'exo_cals_nets', 'Masse_Totale', 'Masse_Grasse']].to_numpy()        
        ds_inputs1 = tf.keras.utils.timeseries_dataset_from_array(
            data=data_inputs1,
            targets=None,
            sequence_length=self.past,
            batch_size=None
        )
        
        # b) all values calories in the FUTURE : columns 'Calories' and 'exo_cals_nets' from i+PAST to i+PAST+FUTURE-1
        data_inputs2 = dataframe[['Calories', 'exo_cals_nets']].to_numpy()
        ds_inputs2 = tf.keras.utils.timeseries_dataset_from_array(
            data=data_inputs2,
            targets=None,
            sequence_length=self.future,
            start_index=self.past,
            batch_size=None
        )             
                
        # # create the output data, by flattening a window of future data
        # imax = len(data_inputs1) - self.past - self.future # maximum indice from which to extract a window of length past+future
        # outputs_raw = dataframe[['Masse_Totale', 'Masse_Grasse']]
        # targets = np.zeros(shape=(imax+1,self.future * 2))
        # for i in range(len(targets)):
        #     targets[i] = outputs_raw.iloc[i+self.past:i+self.past+self.future,:].to_numpy().reshape(-1)
        # data_inputs1 = data_inputs1[:imax+1]
        
        # # create the dataset itself
        # ds_outputs = tf.keras.utils.timeseries_dataset_from_array(
        #     data=targets,
        #     targets=None,
        #     sequence_length=self.future,
        #     batch_size=None
        # )
        
        outputs = dataframe[['Masse_Totale', 'Masse_Grasse']].to_numpy()
        ds_outputs = tf.keras.utils.timeseries_dataset_from_array(
            data=outputs,
            targets=None,
            sequence_length=self.future,
            start_index=self.past,
            batch_size=None
        )
        
        # final dataset
        ds_inputs = tf.data.Dataset.zip((ds_inputs1, ds_inputs2))
        ds = tf.data.Dataset.zip((ds_inputs, ds_outputs))
        
        # the final outputs is a tf.dataset of elements of specs :
        #         ((TensorSpec(shape=(None, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None)),  # INPUTS = PAST values of 4 features (exo_cals_nets, Calories, masse_totale, masse_grasse) + FUTURE values of 2 features (Calories, exo_cals_nets)
        #           TensorSpec(shape=(None, 6), dtype=tf.float64, name=None))                                                               # OUTPUTS = FUTURE values of 2 features (masse_totale, masse_grasse)
        return ds

In [251]:
essaids = CreateDataset()

In [252]:
essaids.dataframe.head(20)

,Calories,exo_cals_nets,Masse_Totale,Masse_Grasse
2020-08-09,1861.3,0.0,107.250000,37.190000
2020-08-11,1772.4,0.0,104.650000,33.150000
2020-08-12,1894.3,0.0,104.350000,33.800000
2020-08-13,1822.6,0.0,104.400000,34.340000
2020-08-14,1891.5,0.0,104.500000,33.850000
2020-08-15,1925.3,0.0,104.250000,34.280000
2020-08-16,2021.7,0.0,104.050000,34.430000
2020-08-17,1946.5,0.0,103.900000,33.880000
2020-08-18,1789.5,0.0,103.550000,34.130000
2020-08-19,1914.8,0.0,103.250000,32.650000


In [253]:
dsf = essaids.create_dataset()

In [254]:
for e in dsf.take(1):
    print(e)

((<tf.Tensor: shape=(7, 4), dtype=float64, numpy=
array([[1861.3 ,    0.  ,  107.25,   37.19],
       [1772.4 ,    0.  ,  104.65,   33.15],
       [1894.3 ,    0.  ,  104.35,   33.8 ],
       [1822.6 ,    0.  ,  104.4 ,   34.34],
       [1891.5 ,    0.  ,  104.5 ,   33.85],
       [1925.3 ,    0.  ,  104.25,   34.28],
       [2021.7 ,    0.  ,  104.05,   34.43]])>, <tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[1946.5,    0. ],
       [1789.5,    0. ],
       [1914.8,    0. ]])>), <tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[103.9 ,  33.88],
       [103.55,  34.13],
       [103.25,  32.65]])>)
